In [1]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import numpy as np
import pandas as pd
from random import choices
import pickle
import sklearn
from sklearn.metrics import roc_auc_score

In [2]:
SEED = 1111

tf.random.set_seed(SEED)
np.random.seed(SEED)

In [3]:
def load_data():
    
    return pickle.load(open('df_down_sampled_alternative.p', 'rb'))

In [174]:
#Function for splitting data into train/test set!

def train_test_split(test_share, data):
    
    #Split data into initial train/test
    
    train_share = 1 - test_share    
    train_size = int(len(data) * train_share)
    train_set = data[0:train_size]
    test_set = data[train_size:len(data)]    
    
    
    return (train_set, test_set)

In [175]:
def create_mlp(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape = (num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)

    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile( 
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name = "AUC"),
    )

    return model

In [176]:
#Load downsample frame! Zero-weight observations and data < 86 have already been removed!

df = load_data()

In [177]:
df.fillna(df.mean(), inplace = True)

In [178]:
df['action'] = ((df['resp'].values) > 0).astype(int)

In [179]:
features = [c for c in df.columns if "feature" in c]

In [180]:
f_mean = np.mean(df[features[1:]].values,axis=0)

In [181]:
resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

In [182]:
train_set, test_set = train_test_split(test_share = 0.3, data = df)

In [183]:
y_train = np.stack([(train_set[c] > 0).astype('int') for c in resp_cols]).T

In [184]:
X_train = train_set.loc[:, train_set.columns.str.contains('feature')]

In [185]:
y_test = np.stack([(test_set[c] > 0).astype('int') for c in resp_cols]).T

In [186]:
X_test = test_set.loc[:, test_set.columns.str.contains('feature')]

In [4]:
X_test

NameError: name 'X_test' is not defined

In [187]:
batch_size = 5000
hidden_units = [150, 150, 150]
dropout_rates = [0.2, 0.2, 0.2, 0.2]
label_smoothing = 1e-2
learning_rate = 1e-3

In [188]:
clf = create_mlp(
    len(features), 5, hidden_units, dropout_rates, label_smoothing, learning_rate
    )

In [189]:
clf.fit(X_train, y_train, epochs = 200, batch_size=5000)

Epoch 1/200
220/220 [==============================] - 26s 108ms/step - loss: 0.7131 - AUC: 0.5144
Epoch 2/200
220/220 [==============================] - 24s 111ms/step - loss: 0.6922 - AUC: 0.5321
Epoch 3/200
220/220 [==============================] - 23s 106ms/step - loss: 0.6906 - AUC: 0.5377
Epoch 4/200
220/220 [==============================] - 26s 116ms/step - loss: 0.6900 - AUC: 0.5412
Epoch 5/200
220/220 [==============================] - 26s 120ms/step - loss: 0.6896 - AUC: 0.5439
Epoch 6/200
220/220 [==============================] - 26s 120ms/step - loss: 0.6894 - AUC: 0.5445
Epoch 7/200
220/220 [==============================] - 26s 118ms/step - loss: 0.6892 - AUC: 0.5458
Epoch 8/200
220/220 [==============================] - 26s 118ms/step - loss: 0.6889 - AUC: 0.5474
Epoch 9/200
220/220 [==============================] - 28s 129ms/step - loss: 0.6889 - AUC: 0.5475
Epoch 10/200
220/220 [==============================] - 32s 145ms/step - loss: 0.6886 - AUC: 0.5491
Epoch 11/

220/220 [==============================] - 25s 116ms/step - loss: 0.6831 - AUC: 0.5666
Epoch 84/200
220/220 [==============================] - 26s 116ms/step - loss: 0.6830 - AUC: 0.5673
Epoch 85/200
220/220 [==============================] - 26s 117ms/step - loss: 0.6831 - AUC: 0.5663
Epoch 86/200
220/220 [==============================] - 25s 115ms/step - loss: 0.6831 - AUC: 0.5667
Epoch 87/200
220/220 [==============================] - 25s 116ms/step - loss: 0.6829 - AUC: 0.5677
Epoch 88/200
220/220 [==============================] - 25s 115ms/step - loss: 0.6830 - AUC: 0.5668
Epoch 89/200
220/220 [==============================] - 25s 113ms/step - loss: 0.6828 - AUC: 0.5678
Epoch 90/200
220/220 [==============================] - 26s 116ms/step - loss: 0.6829 - AUC: 0.5672
Epoch 91/200
220/220 [==============================] - 26s 118ms/step - loss: 0.6827 - AUC: 0.5673
Epoch 92/200
220/220 [==============================] - 25s 114ms/step - loss: 0.6830 - AUC: 0.5673
Epoch 93/200


In [190]:
pickle.dump(clf, open('tf_model.csv','wb'))

TypeError: can't pickle weakref objects

In [ ]:
models = []

models.append(clf)
th = 0.5000

In [ ]:
pred = models[0].predict(X_test)

In [ ]:
pred = (pred > 0.5).astype(int)

In [ ]:
result = (pred == y_test).astype(int)

In [ ]:
np.mean(result)

In [ ]:
pd.DataFrame(result).mean()

In [ ]:
print("Finished!")